In [23]:
from TwitterSearch import *
import datetime
import pandas as pd 
import time
import numpy as np
import re 

# ref:

for i in range(0,7):
    try:
        tso = TwitterSearchOrder() # create a TwitterSearchOrder object
        #tso.set_keywords([])

        words = ['MarchForOurLives','#MarchForOurLivesdc', '#MarchForOurLivesLA', '#MarchForOurLivesBoston',
        '@AMarch4OurLives',"#marchforourlives",'@AMarchForOurLives']

        tso.set_keywords([words[i]]) 
        # let's define all words we would like to have a look for
        tso.set_language('en') # we want to see English tweets only
        tso.set_include_entities(False) # and don't give us all those entity information


        # it's about time to create a TwitterSearch object with our secret tokens
        ts = TwitterSearch(
            consumer_key = '8xOVQj0CndBQQrE3ag3ck5R2k',
            consumer_secret = 'WzJgzSUjPBBONjInO6N4TSSL9eXEGi0UxfftYZywbqGbZaZRkr',
            access_token = '961749540079620096-pfb1dsevjN9Fitj9dn0Vo0qDwcMHuDG',
            access_token_secret = 'ZcjVebxhlAuNey9GX5MZhv7VPzFeNCnkf09sZ41vlELq5'
         )

         # this is where the fun actually starts :)
        i=0

        tweets = []
        for tweet in ts.search_tweets_iterable(tso):
            i = i+1
            if(i%100==0):
                print (i)
            #print( '%s=>%s' % (tweet['created_at'], tweet['text']) )
            row = [tweet['id'] , tweet['created_at'], tweet['text']] 
            #print(row)
            for i in range (len(row)):
                row[i] = re.sub(',',' ',str(row[i]))
            tweets.append(row)
            
        if(len(tweets) > 0):
            print(len(tweets))
            dfTweets = pd.DataFrame(tweets)
            existing = pd.read_csv('Twitterdata.csv')
            x = np.vstack([dfTweets.values, existing.values])
            combined = (pd.DataFrame(x)).drop_duplicates()
            combined.to_csv ('Twitterdata.csv', index = False)

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)


188
1198
700
300
866
188


In [25]:
import requests
import json
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import time
import datetime

def scrapeHTML(url):
    text = ""
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,"html")
    heading = soup.h1.string
    contents = soup.find_all("p")
    for content in contents:
        text = text+str(content.string)+" "
    heading = re.sub(',' , ' ', heading)
    text = re.sub(',' , ' ', text)
    Page = {'url':url, 'content':heading+ " " + text}
    return Page

Pages = pd.DataFrame(columns = ['url','content'])


api_key ="3a2aeebff29144ab99ef0fe96f89ff24"
query = "March for our lives"
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key="+api_key+"&q="+query
response = requests.get(url)
response_data = response.content
json_data =response_data.decode('utf8') # converting data to string
#print(json_data);
#ref => https://docs.python.org/3/library/json.html
data = json.loads(json_data);
#ref => https://docs.python.org/3/library/json.html  
s = json.dumps(data, indent = 2, )


data_dump = data['response']['docs']
len(data_dump)

for i in data_dump:
    Pages = Pages.append(scrapeHTML(i['web_url']), ignore_index = True)
    
old = pd.read_csv('Articles1.csv', encoding = 'latin-1')
xx = np.vstack ([old.values,Pages.values])
ArtCombined = (pd.DataFrame(xx, columns = ['url', 'content'])).drop_duplicates()
ArtCombined.to_csv('Articles1.csv',index=False)



/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [27]:
def uniquify():
    # makes the 2 csvs unique
    x = pd.read_csv('Articles1.csv', encoding = 'latin-1')
    x = x.drop_duplicates(['url'])
    x.to_csv('Articles2.csv')

    y = pd.read_csv('Twitterdata.csv')
    y = y.drop_duplicates(['2'])
    y.to_csv('UniqueTwitterdata.csv')
    
    #print(x['content'], y['2'])
    np.savetxt(r'tweets_final.txt', y['2'].values, fmt='%s')
    np.savetxt(r'articles_final.txt', x['content'].values, fmt='%s')

uniquify()